In [2]:
from ultralytics import YOLO
import cv2
import torch
import numpy as np
import os

In [3]:
def segmentation_to_bbox(masks):
    segmentation = np.where(masks)

    # Bounding Box
    bbox = 0, 0, 0, 0
    if len(segmentation) != 0 and len(segmentation[1]) != 0 and len(segmentation[0]) != 0:
        x_min = int(np.min(segmentation[1]))
        x_max = int(np.max(segmentation[1]))
        y_min = int(np.min(segmentation[0]))
        y_max = int(np.max(segmentation[0]))

        bbox = x_min, x_max, y_min, y_max

    return bbox


def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

def expand_box(boxPoints, n):
    # Calculate the center point of the box
    center_x = int(np.mean(boxPoints[:, 0]))
    center_y = int(np.mean(boxPoints[:, 1]))

    # Create an empty list to store the expanded points
    expanded_points = []

    # Loop over each point and expand it outward from the center by n pixels
    for point in boxPoints:
        # Calculate the vector from the center to the point
        vec_x = point[0] - center_x
        vec_y = point[1] - center_y

        # Calculate the expanded point
        expanded_x = center_x + vec_x * (1 + n)
        expanded_y = center_y + vec_y * (1 + n)

        # Add the expanded point to the list
        expanded_points.append((expanded_x, expanded_y))

    # Convert the list of points to NumPy array
    expanded_points = np.array(expanded_points).astype(int)

    return expanded_points


In [4]:
model = YOLO('2-july-224.pt')

In [5]:
filepath = "Data Train for BDC 2023 - Penyisihan/" + "DataTrain539.png"
# filepath = "output_bright.jpg"

results = model(filepath, imgsz=224)
img = cv2.imread(filepath)

for result in results:
    for mask in result.masks:
        dim = mask.data.shape
        img = cv2.resize(img, (dim[2], dim[1]))

        m = torch.squeeze(mask.data)
        composite = torch.stack((m, m, m), 2)
        tmp = img * composite.cpu().numpy().astype(np.uint8)
        crop = tmp
        cv2.imshow("original", img)
        cv2.imshow("yolo_crop", tmp)
        # cv2.waitKey(0)
        break
    break
# cv2.destroyAllWindows()

cropped = crop.copy()
gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)


contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = max(contours, key=cv2.contourArea)
epsilon = 0.02 * cv2.arcLength(contour, True)
approx = cv2.approxPolyDP(contour, epsilon, True)

perspective_parallelogram = []
# rect = approx
if len(approx) == 4:
    for i in range(len(approx)):
        point_before = approx[i % len(approx)][0]
        perspective_parallelogram.append((point_before[0], point_before[1]))
        # point_after = approx[(i + 1) % len(approx)][0]
    
        # # draw point and line in image
        # cv2.circle(crop, (point_before[0], point_before[1]), 5, (0, 0, 255), -1)
        # cv2.line(crop, (point_before[0], point_before[1]), (point_after[0], point_after[1]), (0, 255, 0), 2)

threshold_ratio = 0.5
area = cv2.contourArea(approx)
rect = cv2.minAreaRect(approx)
bounding_rect_area = cv2.contourArea(cv2.boxPoints(rect))
ratio = area / bounding_rect_area
if ratio > threshold_ratio:
    cv2.drawContours(crop, [approx], -1, (0, 255, 0), 2) 
    box = cv2.boxPoints(cv2.minAreaRect(approx))
    box = np.intp(box)
    cv2.drawContours(crop, [box], 0, (0, 0, 255), 2)

    # expand box
    box = expand_box(box, 0.1)
    cv2.drawContours(crop, [box], 0, (255, 255, 0), 2)

cv2.imshow("box bounded", crop)

# perspective = order_points(np.array(perspective))
if len(perspective_parallelogram) == 4:
    print(perspective_parallelogram)
    perspective = order_points(np.array(perspective_parallelogram))
else:
    perspective = order_points(np.array(box))

height, width, channels = img.shape

pts1 = np.float32(perspective)
pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)
transformed = cv2.warpPerspective(img, matrix, (width, height))

# keep original ratio of license plate
(x, y), (width, height), angle = rect
aspect_ratio = min(width, height) / max(width, height)
transformed = cv2.resize(transformed, (224, int(224 * aspect_ratio)))

# gray = cv2.cvtColor(scaled, cv2.COLOR_BGR2GRAY)

# sharpen image
sharpen_filter=np.array(
    [
        [-1,-1,-1],
        [-1,9,-1],
        [-1,-1,-1]
    ])
transformed = cv2.filter2D(gray,-1,sharpen_filter)

# invert
inverted = cv2.bitwise_not(transformed)
cv2.imshow("inverted", inverted)

# display the image
cv2.imshow("transformed", transformed)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Train for BDC 2023 - Penyisihan\DataTrain539.png: 96x224 1 License Plate, 82.5ms
Speed: 1.0ms preprocess, 82.5ms inference, 7.0ms postprocess per image at shape (1, 3, 224, 224)


[(4, 13), (7, 60), (183, 82), (188, 35)]


NameError: name 'exposure' is not defined

## Old Way

In [6]:
# cropped = crop.copy()
# gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
# gray = cv2.GaussianBlur(gray, (3, 3), 0)
# canny = cv2.Canny(gray, 20, 50)
# kernel = np.ones((3, 3), np.uint8)
# dilation = cv2.dilate(canny, kernel, iterations=2)
# erosion = cv2.erode(dilation, kernel, iterations=1)


# cv2.imshow("erosion", erosion)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [7]:
_, thresh = cv2.threshold(erosion, 45, 255, cv2.THRESH_BINARY)
cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)

NameError: name 'erosion' is not defined

In [ ]:
# find biggest contour
c = max(cnts, key=cv2.contourArea)

peri = cv2.arcLength(c, True)
approx = cv2.approxPolyDP(c, 0.02 * peri, True)

In [ ]:
circle = cropped.copy()
for i in range(len(approx)):
    cv2.circle(circle, (approx[i][0][0], approx[i][0][1]), 3, (0, 255, 0), -1)

cv2.imshow("circle", circle)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
perspective = []
rect = approx
for i in range(len(approx)):
    point_before = approx[i % len(approx)][0]
    perspective.append((point_before[0], point_before[1]))
    point_after = approx[(i + 1) % len(approx)][0]
    cv2.line(cropped, point_before, point_after, (0, 255, 0), 2)

perspective = order_points(np.array(perspective))

cv2.imshow("original", img)
cv2.imshow("cropped", cropped)

height, width, channels = cropped.shape

# pts1 = np.float32([[0, 0], [width - 150, 0], [width, height], [0, height]])
pts1 = np.float32(perspective)
pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)
transformed = cv2.warpPerspective(cropped, matrix, (width, height))

cv2.imshow("transformed", transformed)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Pipelining

In [6]:
unprocessed_images = []

def predict(filepath, model):
    return model.predict(filepath, imgsz=224, verbose=False)

def pipeline_stage1(filepath, target_dir, model):
    results = predict(filepath, model)
    img = cv2.imread(filepath)

    for result in results:
        if result.masks is None:
            result_path = os.getcwd() + "\\" + target_dir + "\\" + os.path.basename(filepath)
            print("WARNING: No mask found for image " + os.path.basename(filepath) + ". Saving original image.")
            unprocessed_images.append(os.path.basename(filepath))
            cv2.imwrite(result_path, img)
            return

        for mask in result.masks:
            dim = mask.data.shape
            img = cv2.resize(img, (dim[2], dim[1]))

            m = torch.squeeze(mask.data)
            composite = torch.stack((m, m, m), 2)
            tmp = img * composite.cpu().numpy().astype(np.uint8)
            crop = tmp
            break
        break


    cropped = crop.copy()
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)


    contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    perspective_parallelogram = []
    # rect = approx
    if len(approx) == 4:
        for i in range(len(approx)):
            point_before = approx[i % len(approx)][0]
            perspective_parallelogram.append((point_before[0], point_before[1]))
            # point_after = approx[(i + 1) % len(approx)][0]
        
            # # draw point and line in image
            # cv2.circle(crop, (point_before[0], point_before[1]), 5, (0, 0, 255), -1)
            # cv2.line(crop, (point_before[0], point_before[1]), (point_after[0], point_after[1]), (0, 255, 0), 2)

    threshold_ratio = 0.5
    area = cv2.contourArea(approx)
    rect = cv2.minAreaRect(approx)
    bounding_rect_area = cv2.contourArea(cv2.boxPoints(rect))
    ratio = area / bounding_rect_area
    if ratio > threshold_ratio:
        cv2.drawContours(crop, [approx], -1, (0, 255, 0), 1) 
        box = cv2.boxPoints(cv2.minAreaRect(approx))
        box = np.intp(box)
        # cv2.drawContours(crop, [box], 0, (0, 0, 255), 2)

        # expand box
        box = expand_box(box, 0.1)
        cv2.drawContours(crop, [box], 0, (255, 255, 0), 2)

    # perspective = order_points(np.array(perspective))
    if len(perspective_parallelogram) == 4:
        perspective = order_points(np.array(perspective_parallelogram))
    else:
        perspective = order_points(np.array(box))

    height, width, channels = img.shape

    pts1 = np.float32(perspective)
    pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    transformed = cv2.warpPerspective(img, matrix, (width, height))

    # keep original ratio of license plate
    (x, y), (width, height), angle = rect
    aspect_ratio = min(width, height) / max(width, height)
    transformed = cv2.resize(transformed, (224, int(224 * aspect_ratio)))

    os.makedirs(target_dir, exist_ok=True)

    result_path = os.getcwd() + "\\" + target_dir + "\\" + os.path.basename(filepath)
    cv2.imwrite(result_path, transformed)
    print("Saved " + result_path)

pipeline_stage1("output_bright.jpg", "stage_1", model)

Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\stage_1\output_bright.jpg


In [ ]:
pipeline_stage1("Data Train for BDC 2023 - Penyisihan/DataTrain13.png", "stage_1_sample", model)

Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\stage_1_sample\DataTrain13.png


C:\Users\vitos\AppData\Local\Temp\ipykernel_16984\803823834.py:59: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


In [7]:
# scan raw_data/images directory
# for each image, run pipeline_stage1
# save result to raw_data/stage_1 directory
file_list = os.listdir("Data Train for BDC 2023 - Penyisihan")
for file in file_list:
    if not file.endswith(".png"):
        continue
    pipeline_stage1("Data Train for BDC 2023 - Penyisihan/" + file, "train_data_stage_1", model)

Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain1.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain10.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain100.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain101.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain102.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain103.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain104.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain105.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain106.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain107.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\train_data_stage_1\DataTrain108.png
Saved c:\Users\vitos\Files\Projects\bdc\penyis

In [8]:
file_list = os.listdir("Data Test for BDC 2023 - Penyisihan")
for file in file_list:
    if not file.endswith(".png"):
        continue
    pipeline_stage1("Data Test for BDC 2023 - Penyisihan/" + file, "test_data_stage_1", model)

Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest1.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest10.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest100.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest11.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest12.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest13.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest14.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest15.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest16.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest17.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTest18.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\test_data_stage_1\DataTes